In [1]:
import os
import pandas as pd

/var/folders/y4/c4phbshn5739xq523fgf_cl40000gn/T/ipykernel_44394/3571106454.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [3]:
def convert_string_to_bool(in_string):
    """if string ="" False, else True"""
    if in_string:
        return True
    else:
        return False

def convert_string_to_bool_int(in_string):
    """if string ="" False, else True"""
    if in_string:
        return 1
    else:
        return 0

def convert_bool_string_to_bool_int(in_string):
    """ if string==False -> False, etc."""
    if in_string=="(False,)" or in_string=="False":
        return 0
    elif in_string=="(True,)" or in_string=="True":
        return 1
    else:
        print(type(in_string))
        print(in_string)
        print(in_string[0])
        raise Exception("Error in Transforming BoolString to Int")

def modify_text_prompt(keys_removed):
    """changes to short version"""
    keys = keys_removed.count("+")
    if keys==0:
        return keys_removed
    elif keys>1:
        return "short"
    else:
        return "long"

In [4]:
def load_data(curren_trial_name, filter_flag=False):
    """Loads the data"""
    file_path = get_data_file_path(current_trial_name=curren_trial_name)
    results_data = pd.read_csv(
        os.path.join("..",file_path), 
        converters = {
            "early_stop":convert_string_to_bool_int, 
            "error": convert_string_to_bool_int,
            "done": convert_bool_string_to_bool_int,
            "success": convert_bool_string_to_bool_int,
            "keys_removed": modify_text_prompt
        }
    )
    header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error", "env_idx"]
    clean_results = results_data[header_order]

    # Filters for short, long and react-2 prompts
    if filter_flag: 
        filtering_condition = (clean_results["keys_removed"]=="short") | (clean_results["keys_removed"]=="long") | (clean_results["keys_removed"]=="react-2")
        clean_results = clean_results[ filtering_condition ]
    
    return clean_results

In [5]:
CURRENT_TRIAL_NAME = "v2_eval_20_3"
CURRENT_TRIAL_NAME2 = "v2_eval_20-40"
CURRENT_TRIAL_NAME3 = "v2_eval_60-40"
CURRENT_TRIAL_NAME4 = "v2_eval_100-35"

clean_results1 = load_data(CURRENT_TRIAL_NAME, True)
clean_results2 = load_data(CURRENT_TRIAL_NAME2)
clean_results3 = load_data(CURRENT_TRIAL_NAME3)
clean_results4 = load_data(CURRENT_TRIAL_NAME4)



clean_results = pd.concat([clean_results1,clean_results2,clean_results3, clean_results4])
print(clean_results.shape)

# clean_results = load_data(CURRENT_TRIAL_NAME)

clean_results.head()

(405, 12)


,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error,env_idx
40,gpt-3.5-turbo-0125,react-2,0.0,cool,1,1,0,9,0,0,0,0
41,gpt-3.5-turbo-0125,react-2,0.0,clean,0,1,0,49,20,0,0,1
42,gpt-3.5-turbo-0125,react-2,0.0,clean,0,0,1,23,17,5,0,2
43,gpt-3.5-turbo-0125,react-2,0.0,put,0,1,0,49,32,0,0,3
44,gpt-3.5-turbo-0125,react-2,0.0,clean,0,1,0,49,36,0,0,4


In [46]:
# This is the table that shows results per env_type
out = clean_results.groupby(["model","keys_removed","env_type"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].count().reset_index()["success"]
out["success_std"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].std().reset_index()["success"]
out["done_std"] = clean_results.groupby(["model","keys_removed","env_type"])["done"].std().reset_index()["done"]
out_header_order = ["model", "keys_removed","temperature","env_type","total","success","success_std","done","done_std","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
# Adding a few more scores.
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
out["normalised_total"] = out["total"] / 135

x = out.groupby(["env_type"])
x.head()

,model,keys_removed,temperature,env_type,total,success,success_std,done,done_std,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage,normalised_total
0,gpt-3.5-turbo-0125,long,0.0,clean,31,17,0.505879,25,0.401610,6,735,325,46,0.442177,0.062585,0.548387,0.680000,0.806452,0.229630
1,gpt-3.5-turbo-0125,long,0.0,cool,21,2,0.300793,11,0.511766,10,751,296,83,0.394141,0.110519,0.095238,0.181818,0.523810,0.155556
2,gpt-3.5-turbo-0125,long,0.0,examine,18,5,0.460889,13,0.460889,5,553,290,54,0.524412,0.097649,0.277778,0.384615,0.722222,0.133333
3,gpt-3.5-turbo-0125,long,0.0,heat,23,13,0.506870,20,0.344350,3,521,270,35,0.518234,0.067179,0.565217,0.650000,0.869565,0.170370
4,gpt-3.5-turbo-0125,long,0.0,put,25,5,0.408248,14,0.506623,11,760,379,84,0.498684,0.110526,0.200000,0.357143,0.560000,0.185185
5,gpt-3.5-turbo-0125,long,0.0,puttwo,17,1,0.242536,8,0.514496,9,644,264,82,0.409938,0.127329,0.058824,0.125000,0.470588,0.125926
6,gpt-3.5-turbo-0125,react-2,0.0,clean,31,7,0.425024,27,0.340777,4,1191,479,43,0.402183,0.036104,0.225806,0.259259,0.870968,0.229630
7,gpt-3.5-turbo-0125,react-2,0.0,cool,21,1,0.218218,16,0.436436,5,923,486,41,0.526544,0.044420,0.047619,0.062500,0.761905,0.155556
8,gpt-3.5-turbo-0125,react-2,0.0,examine,18,12,0.485071,14,0.427793,4,335,135,27,0.402985,0.080597,0.666667,0.857143,0.777778,0.133333
9,gpt-3.5-turbo-0125,react-2,0.0,heat,23,13,0.506870,20,0.344350,3,612,265,37,0.433007,0.060458,0.565217,0.650000,0.869565,0.170370


In [7]:
out.to_csv("../results/pandas_output_per_env_type.csv",sep="&",index=False)

In [23]:
print(out[["model", "keys_removed","env_type","total","success"]])

                 model keys_removed env_type  total   success
0   gpt-3.5-turbo-0125         long    clean     31  0.548387
1   gpt-3.5-turbo-0125         long     cool     21  0.095238
2   gpt-3.5-turbo-0125         long  examine     18  0.277778
3   gpt-3.5-turbo-0125         long     heat     23  0.565217
4   gpt-3.5-turbo-0125         long      put     25  0.200000
5   gpt-3.5-turbo-0125         long   puttwo     17  0.058824
6   gpt-3.5-turbo-0125      react-2    clean     31  0.225806
7   gpt-3.5-turbo-0125      react-2     cool     21  0.047619
8   gpt-3.5-turbo-0125      react-2  examine     18  0.666667
9   gpt-3.5-turbo-0125      react-2     heat     23  0.565217
10  gpt-3.5-turbo-0125      react-2      put     25  0.120000
11  gpt-3.5-turbo-0125      react-2   puttwo     17  0.411765
12  gpt-3.5-turbo-0125        short    clean     31  0.516129
13  gpt-3.5-turbo-0125        short     cool     21  0.142857
14  gpt-3.5-turbo-0125        short  examine     18  0.388889
15  gpt-

In [34]:
def create_pivot(variable="success"):# pivot1 = out.pivot(index=["keys_removed"],columns=["env_type"], values=["success"])
    pivot_table = pd.pivot_table(out, index=["keys_removed"],columns=["env_type"], aggfunc={variable:"first"})
    pivot_table.columns = pivot_table.columns.droplevel(0)
    # pivot1.columns.name=None
    piv_table = pivot_table.reset_index()
    return piv_table

piv1 = create_pivot("success")
piv1.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,17,2,5,13,5,1
1,react-2,7,1,12,13,3,7
2,short,16,3,7,11,7,0


In [35]:
pivot_done = create_pivot("done")
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,25,11,13,20,14,8
1,react-2,27,16,14,20,24,14
2,short,19,11,12,18,13,5


In [36]:
pivot_done = create_pivot("num_repetitions")
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,46,83,54,35,84,82
1,react-2,43,41,27,37,16,22
2,short,86,64,46,42,80,75


In [37]:
pivot_done = create_pivot("num_nothing_happens")
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,325,296,290,270,379,264
1,react-2,479,486,135,265,520,175
2,short,284,399,301,226,387,244


In [43]:
pivot_done = create_pivot("success_percentage") 
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,0.548387,0.095238,0.277778,0.565217,0.20,0.058824
1,react-2,0.225806,0.047619,0.666667,0.565217,0.12,0.411765
2,short,0.516129,0.142857,0.388889,0.478261,0.28,0.000000


In [44]:
pivot_done = create_pivot("normalised_nothing") 
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,0.442177,0.394141,0.524412,0.518234,0.498684,0.409938
1,react-2,0.402183,0.526544,0.402985,0.433007,0.471869,0.281804
2,short,0.453674,0.547325,0.582205,0.432122,0.533793,0.427320


In [45]:
pivot_done = create_pivot("normalised_reps") 
pivot_done.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,0.062585,0.110519,0.097649,0.067179,0.110526,0.127329
1,react-2,0.036104,0.044420,0.080597,0.060458,0.014519,0.035427
2,short,0.137380,0.087791,0.088975,0.080306,0.110345,0.131349


In [10]:
pivot2 = pd.pivot_table(out, columns=["env_type"], aggfunc={"total":"first"})
pivot2["keys_removed"] = "total"
piv2 = pivot2.reset_index()
piv2.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,total,31,21,18,23,25,17,total


In [47]:
pivot3 = pd.pivot_table(out, columns=["env_type"], aggfunc={"normalised_total":"first"})
pivot3["keys_removed"] = "total"
piv3 = pivot3.reset_index()
piv3.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,normalised_total,0.22963,0.155556,0.133333,0.17037,0.185185,0.125926,total


In [11]:
new = pd.concat([piv1,piv2])
new = new.drop("index",axis=1)
new.head()

env_type,keys_removed,clean,cool,examine,heat,put,puttwo
0,long,17,2,5,13,5,1
1,react-2,7,1,12,13,3,7
2,short,16,3,7,11,7,0
0,total,31,21,18,23,25,17


In [41]:
# This is the table that shows total results.
out = clean_results.groupby(["model","keys_removed"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
x = out.groupby("keys_removed")
x.head()

,model,keys_removed,temperature,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage
0,gpt-3.5-turbo-0125,long,0.0,135,43,91,44,3964,1824,384,0.460141,0.096872,0.318519,0.472527,0.674074
1,gpt-3.5-turbo-0125,react-2,0.0,135,43,115,20,4784,2060,186,0.430602,0.038880,0.318519,0.373913,0.851852
2,gpt-3.5-turbo-0125,short,0.0,135,44,78,57,3691,1841,393,0.498781,0.106475,0.325926,0.564103,0.577778


In [17]:
out.to_csv("../results/pandas_output_all_results.csv",sep="&",index=False)

In [18]:
# This is the table that shows results per env_id
out = clean_results.groupby(["model","env_idx","keys_removed"]).sum().reset_index()
out_header_order = ["model", "keys_removed","temperature","env_idx","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby("env_idx")
x.head()

,model,keys_removed,temperature,env_idx,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,long,0.0,0,0,0,1,33,21,7
1,gpt-3.5-turbo-0125,react-2,0.0,0,1,1,0,9,0,0
2,gpt-3.5-turbo-0125,short,0.0,0,0,0,1,15,10,5
3,gpt-3.5-turbo-0125,long,0.0,1,0,1,0,49,40,0
4,gpt-3.5-turbo-0125,react-2,0.0,1,0,1,0,49,20,0
...,...,...,...,...,...,...,...,...,...,...
400,gpt-3.5-turbo-0125,react-2,0.0,133,1,1,0,22,5,0
401,gpt-3.5-turbo-0125,short,0.0,133,0,0,1,35,12,5
402,gpt-3.5-turbo-0125,long,0.0,134,0,0,1,43,12,5
403,gpt-3.5-turbo-0125,react-2,0.0,134,0,1,0,49,21,0


In [19]:
out.pivot(index=["env_idx"],columns=["keys_removed"], values=["success"])

success              
keys_removed    long react-2 short
env_idx                           
0                  0       1     0
1                  0       0     1
2                  1       0     0
3                  0       0     1
4                  0       0     0
...              ...     ...   ...
130                1       0     1
131                0       1     1
132                0       0     0
133                0       1     0
134                0       0     0

[135 rows x 3 columns]

In [48]:
x= pd.Series([0.22963,0.155556,0.133333,0.17037,0.185185,0.125926])
x

0    0.229630
1    0.155556
2    0.133333
3    0.170370
4    0.185185
5    0.125926
dtype: float64

In [53]:
x*20

0    4.59260
1    3.11112
2    2.66666
3    3.40740
4    3.70370
5    2.51852
dtype: float64